# 雷达数据

## 雷达数据的绘制和上传redis




In [1]:
%%time
%matplotlib inline
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import cinrad
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from matplotlib import cm, colors
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from ncmaps import Cmaps
from io import BytesIO
import base64
import json
class nlcmap(LinearSegmentedColormap):
    """A nonlinear colormap"""

    name = 'nlcmap'

    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels/ self.levels.max()
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self._y = np.linspace(self.levmin, self.levmax, len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self._y)
        return self.cmap(yi/self.levmax, alpha)
class radar_data:
    def __init__(self):
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
    def to_redis(self,imd):
        if self.rs.get("radar"):
            data = pickle.loads(self.rs.get("radar"))
            data['imglist'].append(imd)
        else:
            imglist = [imd]
            data = {
                "imglist":imglist
            }
        self.rs.set("radar", pickle.dumps(data))       
    def plot_data(self):
        path ="/home/workspace/Data/My_Git/web_met/static/data/downfile/" 
        f = cinrad.io.CinradReader(path+'Z_RADR_I_Z9576_20150809120400_O_DOR_SA_CAP.bin.bz2')
        tilt_number = 0
        data_radius = 230
        data_dtype = 'REF' # stands for reflectivity
        # 数据加载核心
        ra = f.get_data(tilt_number, data_radius, data_dtype)
        rl = list(f.iter_tilt(230, 'REF'))
        cr = cinrad.easycalc.quick_cr(rl)
        data = ra.data
        m = Basemap(llcrnrlon=119.16,llcrnrlat=26.55,urcrnrlon=123.88,urcrnrlat=30.69)
        ## 颜色 ------
        colorslist = ['#00C800','#019000','#FFFF00','#E7C000','#FF9000','#D60000','#C00000','#FF00F0','#780084','#AD90F0','#AE0AF5']# 组合反射率
        levels1 = [15,20,25,30,35,40,45,50,55,60,65,70]
        cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=11)
        print("-----------",cr.data)
        plt.contourf(cr.lon, cr.lat, cr.data,cmap=cmaps,levels = levels1)
        plt.axis('off') 
        buffer = BytesIO()
        plt.savefig(buffer,bbox_inches='tight',transparent=True)  
        plot_img = buffer.getvalue()
        imb = base64.b64encode(plot_img) 
        ims = imb.decode()
        imd = "data:image/png;base64,"+ims
        self.to_redis(imd)
    def get_radar(self):
        data = pickle.loads(self.rs.get("radar"))
        return data

worker = radar_data()
worker.plot_data()
#data = worker.get_radar()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/cinrad/io/level2.py:355: UserWarning: Requested data range exceed max range in this tilt
  warnings.warn("Requested data range exceed max range in this tilt")


----------- [[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]]
CPU times: user 26.5 s, sys: 1.17 s, total: 27.7 s
Wall time: 3.46 s


In [6]:
data = worker.get_radar()

In [7]:
len(data['imglist'])

12